In [1]:
clean_up = True
%run StdPackages.ipynb
d['gams'] = os.path.join(d['CGE'],'gams')

The file_gams_py_gdb0.gdx is still active and was not deleted.


## Example A:
1. Define global settings that are applied across all modules.
2. Define production structure using nesting trees.
3. Define production module from steps 1-2.
4. Initialize and compile:
    1. Initialize variables (if needed),
    2. Initialize groups definition (add to ```self.s.groups```)
    3. Define argument defining blocks of equations.
    4. Specify settings for different states of the model

*1. Globals:*

Start by defining the global settings: In this section, we draw on the ```SmallOpen``` type. Beyond some global parameters (e.g. interest rate, long run growth rate, etc.), this includes some definitions for time:

In [2]:
glob = CGE_globals.SmallOpen(kwargs_vals = {'t': range(1,5)})

*2. Nesting tree:*

We consider a two-sector model $(s1,s2)$ that produces three goods $(a,b,c)$ using inputs $(K,L,M)$. Sector 1 produces  a single using a nested CES, sector 2 produces two outputs using a nested CES/MNL type function.

In [3]:
name = 'V1'
data_str = os.path.join(d['data'],'ProductionModule_v1.xlsx')
read_trees = {'Tree1': {'f':'CES'}, 'Tree2': {'f': 'MNL_out'}} # Keys refer sheet, f refers to type.
Tree = NestingTree.AggTree_from_data(data_str, read_trees = read_trees, name = name)() # apply call function.

*3. Initialize production module from tree*

In [4]:
P = CGE_Production.Production(tree = Tree, glob = glob)

#### 4: Initialize and compile

The method ```self.compile(order=None, initDB=False)``` is defined for all ```GmsPython``` instances, of which ```CGE_Production.Production``` is a specific instance. This method works through two/three steps:
1. ```self.compile_groups:``` Updates the settings ```self.s``` specification of groups from the method ```self.groups()```. A standard method for ```self.groups``` collects ```self.groups()``` for all the modules in ```self.m``` that has such a method defined. The production module has a specific method implemented here. Next, the groups are compiled using ```self.s.Compile.run()```.
2. ```self.compile_args:``` Collects and sorts 'args' for the model stored in ```self.s['args']``` specificied by the method ```self.args()```. A standard method ```self.args``` collects ```self.args()``` for all the modules in ```self.m``` that has such a method defined. The production module has a specific method implemented here.
3. ```self.initDB```: If ```initDB=True``` a method that supplies simple initial values for relevant variables are applied. A standard method is applied that collects ```self.initDB``` methods from the modules in ```self.m```. 

*The following can be run by calling ```self.compile(initDB=True)```.*

*1. Compile groups: This defines and compile the groups.*

In [5]:
P.compile_groups()

{'G_V1_exo_always': <_GmsPy.Group at 0x1ddd9631e20>,
 'G_V1_endo_always': <_GmsPy.Group at 0x1ddd963c460>,
 'G_V1_exo_in_calib': <_GmsPy.Group at 0x1ddd963c4c0>,
 'G_V1_endo_in_calib': <_GmsPy.Group at 0x1ddd963c040>}

*Once collected, note importantly that we can access conditions on variables for each group:*

In [6]:
conditions = P.s.Compile.groups['G_V1_exo_always'].conditions
conditions

{'qS': <_Database.gpy at 0x1ddd96310d0>,
 'pD': <_Database.gpy at 0x1ddd4988c40>,
 'sigma': <_Database.gpy at 0x1ddd4988fa0>}

*These are conditions that can be used to subset pandas objects. In other words, we can access a pandas representation of e.g. 'sigma' that belongs to the specific group:*

In [7]:
rc_pd(P.get('sigma'), c = conditions['sigma'])

s   n 
s1  KL    0.5
    a       2
s2  Y     0.5
Name: sigma, dtype: object

*2. Compile args: This defines some arguments that are eventually written to gams code.*

In [8]:
P.compile_args()

{'V1_blocks': '\n$BLOCK B_V1_Tree1\n\tE_zp_out_Tree1[t,s,n]$(knot_o_Tree1[s,n] and txE[t])..\tpS[t,s,n]*qS[t,s,n] =E= sum(nn$(map_Tree1[s,n,nn]), qD[t,s,nn]*pD[t,s,nn]);\n\tE_zp_nout_Tree1[t,s,n]$(knot_no_Tree1[s,n] and txE[t])..\tpD[t,s,n]*qD[t,s,n] =E= sum(nn$(map_Tree1[s,n,nn]), qD[t,s,nn]*pD[t,s,nn]);\n\tE_q_out_Tree1[t,s,n]$(branch2o_Tree1[s,n] and txE[t])..\tqD[t,s,n] =E= sum(nn$(map_Tree1[s,nn,n]), mu[s,nn,n] * (pS[t,s,nn]/pD[t,s,n])**(sigma[s,nn]) * qS[t,s,nn]);\n\tE_q_nout_Tree1[t,s,n]$(branch2no_Tree1[s,n] and txE[t])..\tqD[t,s,n] =E= sum(nn$(map_Tree1[s,nn,n]), mu[s,nn,n] * (pD[t,s,nn]/pD[t,s,n])**(sigma[s,nn]) * qD[t,s,nn]);\n$ENDBLOCK\n\n\n$BLOCK B_V1_Tree2\n\tE_zp_Tree2[t,s,n]$(knot_Tree2[s,n] and txE[t])..\tpD[t,s,n]*qD[t,s,n] =E= sum(nn$(map_Tree2[s,n,nn] and branch_o_Tree2[s,nn]), qS[t,s,nn]*pS[t,s,nn])+sum(nn$(map_Tree2[s,n,nn] and branch_no_Tree2[s,nn]), qD[t,s,nn]*pD[t,s,nn]);\n\tE_q_out_Tree2[t,s,n]$(branch_o_Tree2[s,n] and txE[t])..\tqS[t,s,n] =E= sum(nn$(map_Tree